<a href="https://colab.research.google.com/github/pepperamy/others/blob/main/get_sp500_post_fl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time

#### get the accounts from S&P_500 list


In [ ]:
list_account_res = []
params = (('token', '6inSCSaD5HTd1tdYEWGESSxduwgMF517gNrCJU1c'),('count',100))
list_account = requests.get('https://api.crowdtangle.com/lists/1527924/accounts',\
                            params=params)
list_account = list_account.json()
list_account_res += list_account['result']['accounts']
p = 0
while ('pagination' in list_account['result'].keys()) and ('nextPage'in list_account['result']['pagination'].keys()):
    p += 1
    print(p)
    list_account = requests.get(list_account['result']['pagination']['nextPage'])
    list_account = list_account.json()
    list_account_res += list_account['result']['accounts'] 

1
2
3
4


In [ ]:
list_account = pd.DataFrame(list_account_res)

In [ ]:
list_account.shape

(444, 11)

In [ ]:
list_account.head()

,id,name,handle,profileImage,subscriberCount,url,platform,platformId,accountType,pageAdminTopCountry,verified
0,1864990,"Analog Devices, Inc.",AnalogDevicesInc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,133076,https://www.facebook.com/243348722355011,Facebook,243348722355011,facebook_page,US,False
1,149249,Ameriprise Financial,Ameriprise,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,132970,https://www.facebook.com/161275430585313,Facebook,161275430585313,facebook_page,US,True
2,1066260,Bristol Myers Squibb,BristolMyersSquibb,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,135030,https://www.facebook.com/474591366067852,Facebook,474591366067852,facebook_page,US,True
3,11090093,Celanese Corporation,Celanese,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,5323,https://www.facebook.com/444667375205,Facebook,444667375205,facebook_page,US,False
4,2264358,Chubb North America,ChubbInsurance,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,21386,https://www.facebook.com/195143962298,Facebook,195143962298,facebook_page,US,False


##### extract the post from api according to the accounts

In [ ]:
#the function to call api
def check_data(x, startdate, enddate):
    fail = None
    res = None
    post = []
    #print('inside',x)
    params = (('token', '6inSCSaD5HTd1tdYEWGESSxduwgMF517gNrCJU1c'),('accounts',x),('count',100),('startDate',startdate),\
             ('endDate',enddate),('sortBy','date'),('searchTerm','covid,coronavirus,pandemic,covid-19,covid19'))
    #note: 'searchTerm' can not do partial search so we add "covid19" "covid-19"
    #If you need to extract all posts for these period, please remove searchTerm
    try:
        response = requests.get('https://api.crowdtangle.com/posts', params=params)
        t = random.randint(0, 30) 
        time.sleep(t)
        if response.status_code==200:
            #print('status',response.status_code)
            res_dict = response.json()
            if 'result' in res_dict.keys():
                res = 1
                #print(len( res_dict['result']['posts']))
                post += res_dict['result']['posts']

            p=0
            while ('pagination' in res_dict['result'].keys()) and ('nextPage'in res_dict['result']['pagination'].keys() ):
                p += 1
                print('page',p,startdate )
                response = requests.get(res_dict['result']['pagination']['nextPage'])
                t = random.randint(0, 30) 
                time.sleep(t)
                res_dict = response.json()
                post += res_dict['result']['posts']

        elif response.status_code !=200:
            print('test error')
            fail = x
        else:
            pass

    except:
        fail = x
    return res, post, fail

In [ ]:
#get a demo table
res = check_data(list_account.platformId[0],'2020-01-01','2020-03-01')

In [ ]:
demo = pd.DataFrame(res[1])

In [ ]:
demo.shape

(12, 21)

In [ ]:
demo.columns

Index(['platformId', 'platform', 'date', 'updated', 'type', 'message',
       'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media',
       'statistics', 'account', 'legacyId', 'id', 'title', 'caption',
       'description', 'imageText', 'videoLengthMS'],
      dtype='object')

In [ ]:
result = pd.DataFrame(columns = list(demo.columns))

In [ ]:
datelist = ['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',\
           '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',\
            '2021-01-01', '2021-02-01', '2021-03-01']

In [ ]:
c_acc = 0
length = 0
fail_list = []

for idx in list_account.index:
    
    print(list_account.platformId[idx])
    
    
    c_acc += 1
    if c_acc%10 == 0:
        print(c_acc)
    idfb = list_account.platformId[idx]
    
    for didx, _ in enumerate(datelist):
        #print(didx)
        if didx >=  len(datelist)-1:
            break

        else:
            res = check_data(idfb, datelist[didx], datelist[didx+1])

            error_c = 0
            if res[2] == None:
                error_c += 1
                if error_c >= len(datelist):
                    fail_list.append(res[2])
                    print('error 2')

            if res[0] == 1:
                #print(res[0])
                temp =  pd.DataFrame(res[1])

                if len(temp) >= 100:
                    print('*****') # for most of the cases, the posts for each account are less than 100

                length += len(temp)

                result = result.append( temp ,ignore_index=True)

In [ ]:
result.shape

(142728, 23)

In [ ]:
result[result.astype(str).duplicated()].index

Int64Index([  1885,   1886,   1887,   1888,   1889,   1890,   1891,   1892,
              1893,   1894,
            ...
            142651, 142652, 142653, 142654, 142657, 142658, 142659, 142662,
            142663, 142664],
           dtype='int64', length=18093)

In [ ]:
result_1 = result.drop(result[result.astype(str).duplicated()].index)

In [ ]:
result_1.shape

(124635, 23)

In [ ]:
df_api.columns

In [ ]:
# DONE